In [1]:
%cd "/content/drive/MyDrive/Car Recognition/Code"

/content/drive/MyDrive/Car Recognition/Code


In [2]:
!pip install tf-nightly

     |████████████████████████████████| 453.4MB 37kB/s 
     |████████████████████████████████| 471kB 38.0MB/s 
     |████████████████████████████████| 4.2MB 25.0MB/s 
     |████████████████████████████████| 1.3MB 34.1MB/s 
     |████████████████████████████████| 4.0MB 30.6MB/s 
     |████████████████████████████████| 5.9MB 33.9MB/s 
     |████████████████████████████████| 4.9MB 29.6MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.37.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: h5py 2.10.0
    Uninstalli

In [3]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json

In [4]:
def load_model(results_folder):
    # load json and create model
    json_file = open(results_folder + '/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(results_folder + "/weights.best.hdf5")
    print("Loaded model from disk")
    return loaded_model

In [5]:
def create_tensorlite(model, weights_path, optimize=False):
  file_tflite = 'model.tflite'
  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  if optimize:
    print('Optimize model tensorflow lite!')
    file_tflite = file_tflite.split('.')[0] + '_optimized.' + file_tflite.split('.')[-1]
    converter.optimizations = [tf.lite.Optimize.DEFAULT] # sử dụng optimization nếu cần tối ưu thêm
  tflite_model = converter.convert()
  with open('%s/%s' %(weights_path, file_tflite), 'wb') as f:
    f.write(tflite_model)


In [7]:
weights_path = '/content/drive/MyDrive/Car Recognition/Code/saved_weights/mobilenet_weights/'
model = load_model(weights_path)
create_tensorlite(model, weights_path, optimize=False)

Loaded model from disk


INFO:tensorflow:Assets written to: /tmp/tmperilojbg/assets


INFO:tensorflow:Assets written to: /tmp/tmperilojbg/assets


In [8]:
interpreter = tf.lite.Interpreter(model_path=weights_path + "model_optimized.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("input details", input_details)
print("ouput details", output_details)

input details [{'name': 'input_1', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
ouput details [{'name': 'Identity', 'index': 100, 'shape': array([  1, 195], dtype=int32), 'shape_signature': array([ -1, 195], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [10]:
import tensorflow as tf
import numpy as np

interpreter = tf.lite.Interpreter(model_path=weights_path + "model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_data = np.random.rand(1, 224, 224, 3).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

boxes = interpreter.get_tensor(output_details[0]['index'])[0]
print(boxes)

[0.01305156 0.00415522 0.00117184 0.00582849 0.00199638 0.00286471
 0.00205758 0.00243375 0.00358418 0.0024165  0.0023361  0.00415041
 0.00658431 0.00291293 0.0045201  0.00807528 0.00483248 0.00092397
 0.00281059 0.0034657  0.00277633 0.00617929 0.00215794 0.00239459
 0.00177257 0.00265083 0.00494918 0.00173123 0.00653039 0.01565104
 0.0005792  0.00510388 0.00233826 0.02657731 0.00944383 0.00456972
 0.00180791 0.00678337 0.00546097 0.00574234 0.00845348 0.0070191
 0.00302604 0.00546431 0.00221272 0.00749672 0.00420167 0.01203099
 0.00171749 0.00117544 0.00129895 0.00128309 0.00157597 0.00377496
 0.00488739 0.00702029 0.00363078 0.01495456 0.00771519 0.00830936
 0.00612332 0.00366774 0.00207309 0.00416598 0.00294605 0.00622245
 0.0024431  0.00232371 0.00158337 0.0023902  0.0020685  0.00339317
 0.00380235 0.00498476 0.01043452 0.00608031 0.01316839 0.00436441
 0.00184516 0.00148838 0.00976765 0.0079493  0.00407366 0.01024806
 0.00476257 0.00512041 0.01048171 0.00753251 0.01587094 0.00562

In [ ]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 112kB 44.5MB/s 
     |████████████████████████████████| 3.8MB 38.0MB/s 
     |████████████████████████████████| 2.9MB 38.8MB/s 
ERROR: tf-nightly 2.6.0.dev20210514 has requirement gast==0.4.0, but you'll have gast 0.3.3 which is incompatible.
ERROR: tf-nightly 2.6.0.dev20210514 has requirement grpcio<2.0,>=1.37.0, but you'll have grpcio 1.32.0 which is incompatible.
ERROR: tf-nightly 2.6.0.dev20210514 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
  Found existing installation: tensorflow-hub 0.12.0
    Uninstalling tensorflow-hub-0.12.0:
      Successfully uninstalled tensorflow-hub-0.12.0
  Found existing installation: grpcio 1.37.1
    Uninstalling grpcio-1.37.1:
      Successfully uninstalled grpcio-1.37.1
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: gast 0.4.

In [ ]:
!tensorflowjs_converter --input_format=keras "/content/drive/MyDrive/mobilenet_weights/weights.best.hdf5" "/content/drive/MyDrive/mobilenet_weights_js/"